In [1]:
import geopandas as gpd
import pandas as pd
import shapely

df = pd.read_csv('/data/Aldhani/eoagritwin/et/data/stations/et_stations.csv', sep=';', encoding="ISO-8859-1")
# List of valid cell IDs
valids = df['FORCE_TILE'].tolist()

# read FORCE_TILES shapefile
gdf = gpd.read_file('/data/Aldhani/dc/deu/vector/datacube-grid_DEU_10km.gpkg', layer='datacube-grid_DEU_10km')
gdf = gdf.to_crs(epsg=4326)

aoiList = []
# Loop through the features
for _, row in gdf.iterrows():
    if row['Tile_ID'] in valids:
        geometry = row.geometry  # Get geometry of the feature
        if geometry.geom_type == 'Polygon':
            coords = list(geometry.exterior.coords)  # Get coordinates of polygon
        else:
            coords = list(geometry.coords)  # For non-polygon geometries

        clist = [c for co in coords for c in co]
        clist.sort()
        aoiList.append({"west": clist[0], "south": clist[5], "east": clist[4], "north": clist[9]})
        

In [ ]:
import openeo
import matplotlib.pyplot as plt
import xarray as xr
import os

connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

connection.describe_collection("SENTINEL3_SLSTR_L2_LST")

dates = [["2024-01-01", "2024-12-31"], ["2023-01-01", "2023-12-31"], ["2022-01-01", "2022-12-31"], ["2021-01-01", "2021-12-31"],
         ["2020-01-01", "2020-12-31"], ["2019-01-01", "2019-12-31"], ["2018-01-01", "2018-12-31"], ["2017-01-01", "2017-12-31"]]

germany = {"west": 5.592041, "south": 47.129951, "east": 15.26001, "north": 55.09723}

for i, aoi in enumerate(aoiList):
    for date in dates:
        storPath = f"/data/Aldhani/eoagritwin/et/data/Sentinel3/{valids[i]}_{date[0]}_{date[1]}.nc"
        if os.path.exists(storPath):
            print(f"{storPath} already exists - next one")
        else:
            try:
                sentinel3_cube = connection.load_collection(
                    "SENTINEL3_SLSTR_L2_LST",
                    spatial_extent = aoi,
                    temporal_extent = date,
                    bands=["LST"],
                )

                cloud_mask = connection.load_collection(
                    "SENTINEL3_SLSTR_L2_LST",
                    temporal_extent = date,
                    spatial_extent = aoi,
                    bands=["confidence_in"],
                )

                cloud_mask = cloud_mask >= 16384

                LST = sentinel3_cube.mask(cloud_mask)
                LST.download(storPath)
            except Exception as e:
                print(e)
                continue

In [ ]:
import openeo
import matplotlib.pyplot as plt
import xarray as xr

connection = openeo.connect("openeo.dataspace.copernicus.eu").authenticate_oidc()

connection.describe_collection("SENTINEL3_SLSTR_L2_LST")

dates = [#["2024-01-01", "2024-12-31"], ["2023-01-01", "2023-12-31"], 
         ["2022-01-01", "2022-12-31"], ["2021-01-01", "2021-12-31"],
         ["2020-01-01", "2020-12-31"], ["2019-01-01", "2019-12-31"], ["2018-01-01", "2018-12-31"], ["2017-01-01", "2017-12-31"]]

dates = [["2022-01-01", "2022-01-31"], ["2022-02-01", "2022-02-28"], ["2022-03-01", "2022-03-31"], 
         ["2022-04-01", "2022-04-30"], ["2022-05-01", "2022-05-31"], ["2022-06-01", "2022-06-30"],
         ["2022-07-01", "2022-07-31"], ["2022-08-01", "2022-08-31"], ["2022-09-01", "2022-09-30"], 
         ["2022-10-01", "2022-10-31"], ["2022-11-01", "2022-11-30"], ["2022-12-01", "2022-12-31"],]

germany = {"west": 5.592041, "south": 47.129951, "east": 15.26001, "north": 55.09723}
germanyList = [{"west": 5.592041, "south": 47.129951, "east": 10, "north": 51},
               {"west": 10, "south": 47.129951, "east": 15.26001, "north": 51},
               {"west": 5.592041, "south": 51, "east": 10, "north": 55.09723},
               {"west": 10, "south": 51, "east": 15.26001, "north": 55.09723}]
for date in dates:
    print(aoi, date)
    try:
        sentinel3_cube = connection.load_collection(
            "SENTINEL3_SLSTR_L2_LST",
            spatial_extent = germany,
            temporal_extent = date,
            bands=["LST"],
        )

        cloud_mask = connection.load_collection(
            "SENTINEL3_SLSTR_L2_LST",
            spatial_extent = germany,
            temporal_extent = date,
            bands=["confidence_in"],
        )

        cloud_mask = cloud_mask >= 16384

        LST = sentinel3_cube.mask(cloud_mask)
        LST.download(f"/data/Aldhani/eoagritwin/et/data/Sentinel3/Germany_{date[0]}_{date[1]}.nc")
    except Exception as e:
        print(e)
        continue

Authenticated using refresh token.
{'west': 13.844073940126027, 'south': 51.786672747838345, 'east': 14.304018041174343, 'north': 52.07086657455895} ['2022-01-01', '2022-01-31']
{'west': 13.844073940126027, 'south': 51.786672747838345, 'east': 14.304018041174343, 'north': 52.07086657455895} ['2022-02-01', '2022-02-28']
{'west': 13.844073940126027, 'south': 51.786672747838345, 'east': 14.304018041174343, 'north': 52.07086657455895} ['2022-03-01', '2022-03-31']
{'west': 13.844073940126027, 'south': 51.786672747838345, 'east': 14.304018041174343, 'north': 52.07086657455895} ['2022-04-01', '2022-04-30']


In [ ]:
ds = xr.load_dataset("LST.nc")
# Select 16 time steps (adjust this as needed)
time_indices = list(range(16))  # Select first 16 time steps
selected_times = ds.isel(t=time_indices)

# Create a 4x4 figure
fig, axes = plt.subplots(4, 4, figsize=(12, 12))

# Flatten the axes array for easy iteration
axes = axes.flatten()

for i, t_index in enumerate(time_indices):
    ax = axes[i]
    data = selected_times.LST.isel(t=i)  # Extract single time slice
    im = ax.imshow(data, cmap="viridis", origin="lower", aspect="auto")
    
    # Add title with time information
    ax.set_title(str(selected_times.t[i].values)[:16], fontsize=8)
    ax.axis("off")  # Hide axis for better visualization

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
job = LST.execute_batch(
    outputfile="lst.nc"
)


In [1]:
dates = [["2021-01-01", "2021-01-31"], ["2021-02-01", "2021-02-28"], ["2021-03-01", "2021-03-31"], 
         ["2021-04-01", "2021-04-30"], ["2021-05-01", "2021-05-31"], ["2021-06-01", "2021-06-30"],
         ["2021-07-01", "2021-07-31"], ["2021-08-01", "2021-08-31"], ["2021-09-01", "2021-09-30"], 
         ["2021-10-01", "2021-10-31"], ["2021-11-01", "2021-11-30"], ["2021-12-01", "2021-12-31"],

         ["2020-01-01", "2020-01-31"], ["2020-02-01", "2020-02-29"], ["2020-03-01", "2020-03-31"], 
         ["2020-04-01", "2020-04-30"], ["2020-05-01", "2020-05-31"], ["2020-06-01", "2020-06-30"],
         ["2020-07-01", "2020-07-31"], ["2020-08-01", "2020-08-31"], ["2020-09-01", "2020-09-30"], 
         ["2020-10-01", "2020-10-31"], ["2020-11-01", "2020-11-30"], ["2020-12-01", "2020-12-31"],

         ["2019-01-01", "2019-01-31"], ["2019-02-01", "2019-02-28"], ["2019-03-01", "2019-03-31"], 
         ["2019-04-01", "2019-04-30"], ["2019-05-01", "2019-05-31"], ["2019-06-01", "2019-06-30"],
         ["2019-07-01", "2019-07-31"], ["2019-08-01", "2019-08-31"], ["2019-09-01", "2019-09-30"], 
         ["2019-10-01", "2019-10-31"], ["2019-11-01", "2019-11-30"], ["2019-12-01", "2019-12-31"],

         ["2018-01-01", "2018-01-31"], ["2018-02-01", "2018-02-28"], ["2018-03-01", "2018-03-31"], 
         ["2018-04-01", "2018-04-30"], ["2018-05-01", "2018-05-31"], ["2018-06-01", "2018-06-30"],
         ["2018-07-01", "2018-07-31"], ["2018-08-01", "2018-08-31"], ["2018-09-01", "2018-09-30"], 
         ["2018-10-01", "2018-10-31"], ["2018-11-01", "2018-11-30"], ["2018-12-01", "2018-12-31"],
         
         ["2017-01-01", "2017-01-31"], ["2017-02-01", "2017-02-28"], ["2017-03-01", "2017-03-31"], 
         ["2017-04-01", "2017-04-30"], ["2017-05-01", "2017-05-31"], ["2017-06-01", "2017-06-30"],
         ["2017-07-01", "2017-07-31"], ["2017-08-01", "2017-08-31"], ["2017-09-01", "2017-09-30"], 
         ["2017-10-01", "2017-10-31"], ["2017-11-01", "2017-11-30"], ["2017-12-01", "2017-12-31"]]

In [2]:
print(len(dates))
del dates[5]
print(len(dates))   

60
59
